# Photoionization of atomic argon: compute cross sections and angular parameters

In [1]:
using JAC

┌ Info: Recompiling stale cache file /home/fritzsch/.julia/compiled/v1.0/JAC/vyEUL.ji for JAC [830ae420-d14d-11e8-2f94-6b071437414d]
└ @ Base loading.jl:1190



Welcome to JAC:  A community approach to the computation of atomic structures, cascades and time evolutions [(C) Copyright by Stephan Fritzsche, Jena (2019)].


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-6738029838641284790\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-13011610384850657034\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-6738029838641284790\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")


Here, we wish to calculate the cross sections and the $\beta_2$ angular parameters for the photoionzation of atomic argon by linearly-polarized synchrotron radiation. Since the ionization process depends on the energy and polarization of the incident radiatian, the **standard settings** (although available be default) are of little help, and at least the photon energies should normally be specified in advance.



Therefore, let us first have a look how these settings are defined internally:


In [2]:
? PhotoIonization.Settings

`struct  PhotoIonization.Settings`  ... defines a type for the details and parameters of computing photoionization lines.

```
+ multipoles              ::Array{EmMultipole}           ... Specifies the multipoles of the radiation field that are to be included.
+ gauges                  ::Array{UseGauge}              ... Specifies the gauges to be included into the computations.
+ photonEnergies          ::Array{Float64,1}             ... List of photon energies.  
+ calcAnisotropy          ::Bool                         ... True, if the beta anisotropy parameters are to be calculated and false otherwise (o/w). 
+ calcPartialCs           ::Bool                         ... True, if partial cross sections are to be calculated and false otherwise.  
+ calcTensors             ::Bool                         ... True, if statistical tensors of the excited atom are to be calculated and false o/w. 
+ printBefore             ::Bool                         ... True, if all energies and lines are printed before their evaluation.
+ selectLines             ::Bool                         ... True, if lines are selected individually for the computations.
+ selectedLines           ::Array{Tuple{Int64,Int64},1}  ... List of lines, given by tupels (inital-level, final-level).
+ stokes                  ::ExpStokes                    ... Stokes parameters of the incident radiation.
```

---

`PhotoIonization.Settings()`  ... constructor for the default values of photoionization line computations

---

`PhotoIonization.Settings(set::PhotoIonization.Settings;`

```
    multipoles=..,          gauges=..,                  photonEnergies=..,          calcAnisotropy=..,    
    calcPartialCs0..,       calcTensors=..,             printBefore=..,             selectLines=..,             
    selectedLines=..,       stokes=..)
                
... constructor for modifying the given PhotoIonization.Settings by 'overwriting' the previously selected parameters.
```


In [3]:
defaultSettings = PhotoIonization.Settings()

multipoles:               EmMultipole[E1]  
gauges:                   UseGauge[UseCoulomb]  
photonEnergies:           Float64[]  
calcAnisotropy:           false  
calcPartialCs:            false  
calcTensors:              false  
printBefore:              false  
selectLines:              false  
selectedLines:            Tuple{Int64,Int64}[]  
stokes:                   exp. Stokes parameters P1 = 0.0,  P2 = 0.0,  P3 = 0.0  



As seen from these settings, the default only includes the E1 photoionization amplitudes, the use of Coulomb gauge, and *no* photon energies are yet defined. 


We here wish to calculate the cross sections from just the Ar $3s^2 3p^6$ $^1S_0$ ground level at the two photon energies 50 eV and 100 eV, respectively. For this, let us re-set the energy units (to eV) and the `Photoionization.Settings` to:


In [4]:
setDefaults("unit: energy", "eV")
e1 = convertUnits("energy: to atomic",  50.)
e2 = convertUnits("energy: to atomic",  100.)

3.674932248085465

In [5]:
photoSettings = PhotoIonization.Settings(defaultSettings, gauges=[UseCoulomb, UseBabushkin], photonEnergies=[e1, e2], 
                                         calcAnisotropy=true, calcPartialCs=true, printBefore=true)

multipoles:               EmMultipole[E1]  
gauges:                   UseGauge[UseCoulomb, UseBabushkin]  
photonEnergies:           [1.83747, 3.67493]  
calcAnisotropy:           true  
calcPartialCs:            true  
calcTensors:              false  
printBefore:              true  
selectLines:              false  
selectedLines:            Tuple{Int64,Int64}[]  
stokes:                   exp. Stokes parameters P1 = 0.0,  P2 = 0.0,  P3 = 0.0  



With these settings, we can proceed along standard lines and compute the cross section and $\beta_2$ parameters by:


In [6]:
wa = Atomic.Computation("Photoionization of Ar: cross sections + beta's",  
            Nuclear.Model(18.);  grid=JAC.Radial.Grid("grid: by given parameters"; rnt = 2.0e-5, 
                                                      h = 5.0e-2, hp = 1.3e-2, NoPoints = 600),
            initialConfigs=[Configuration("[Ne] 2s^2 2p^6")],
            finalConfigs  =[Configuration("[Ne] 2s^2 2p^5")], 
            process = JAC.Photo,  processSettings=photoSettings)
perform(wa)

Define a radial grid of type MeshGL with 610 grid points
 [rnt=2.000e-05, h=5.000e-02, hp=1.300e-02, NoPoints=600, r_max=4.699e+00;
  B-splines with break points at every 7th point, nsL=86, nsS=87, orderL=7, orderS=8, orderGL=7] 

... in perform['computation: SCF'] ...
>> include Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 
(Re-) Define a new standard subshell list.
Start SCF process with hydrogenic orbitals.
(Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 18.0 with mass = 37.62, radius R = 3.37121133160794 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -1.62704537e+02    -1.6270

# **The following part of this nootebook is still under construction.**
What can be seen from this output ?? .. and what else could be controlled ?? ... for this, we refer the user to the manual.
